In [1]:
# imports
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import \
(cross_validate ,
KFold ,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

# 5.3.1 The Validation Set Approach

In [3]:
# load and split data
Auto = load_data('Auto')
Auto_train , Auto_valid = train_test_split(Auto ,
                                        test_size=196,
                                        random_state=0)

In [4]:
# fitting linear regressor
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train , X_train)
results = model.fit()

In [5]:
# manually calculating validation MSE
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [6]:
# function to get validation MSE for higher-degree polynomial fits
def evalMSE(terms ,
            response ,
            train ,
            test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    
    return np.mean((y_test - test_pred)**2)

In [7]:
# apply function on linear, quadratic, cubic fits
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train ,
                        Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [8]:
# results for a different data split
Auto_train , Auto_valid = train_test_split(Auto ,
                                            test_size=196,
                                            random_state=3)
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train ,
                        Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

# 5.3.2 Cross-Validation

In [9]:
# using sklearn_sm wrapper from ISLP
hp_model = sklearn_sm(sm.OLS ,
                      MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model ,
                            X,
                            Y,
                            cv=Auto.shape[0])   # results in LOOCV
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

In [10]:
# function to apply this to polynomial fits
# generate polynomial fits and their results using a for loop
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])   # again LOOCV
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.03322411])

In [12]:
# example of how no.power.outer works
H = np.array(Auto['horsepower'])
H

array([130, 165, 150, 150, 140, 198, 220, 215, 225, 190, 170, 160, 150,
       225,  95,  95,  97,  85,  88,  46,  87,  90,  95, 113,  90, 215,
       200, 210, 193,  88,  90,  95, 100, 105, 100,  88, 100, 165, 175,
       153, 150, 180, 170, 175, 110,  72, 100,  88,  86,  90,  70,  76,
        65,  69,  60,  70,  95,  80,  54,  90,  86, 165, 175, 150, 153,
       150, 208, 155, 160, 190,  97, 150, 130, 140, 150, 112,  76,  87,
        69,  86,  92,  97,  80,  88, 175, 150, 145, 137, 150, 198, 150,
       158, 150, 215, 225, 175, 105, 100, 100,  88,  95,  46, 150, 167,
       170, 180, 100,  88,  72,  94,  90,  85, 107,  90, 145, 230,  49,
        75,  91, 112, 150, 110, 122, 180,  95, 100, 100,  67,  80,  65,
        75, 100, 110, 105, 140, 150, 150, 140, 150,  83,  67,  78,  52,
        61,  75,  75,  75,  97,  93,  67,  95, 105,  72,  72, 170, 145,
       150, 148, 110, 105, 110,  95, 110, 110, 129,  75,  83, 100,  78,
        96,  71,  97,  97,  70,  90,  95,  88,  98, 115,  53,  8

In [17]:
# applies each power to H
X = np.power.outer(H, [1, 2, 3])
X

array([[    130,   16900, 2197000],
       [    165,   27225, 4492125],
       [    150,   22500, 3375000],
       ...,
       [     84,    7056,  592704],
       [     79,    6241,  493039],
       [     82,    6724,  551368]], dtype=int64)

In [18]:
# doing the same as before, but for K < n
cv_error = np.zeros(5)

# defining cross validation function beforehand
cv = KFold(n_splits=10,
          shuffle=True,
          random_state=0)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)   # applying cv
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13719154])

In [19]:
# using shuffle split to achieve validation set approach
validation = ShuffleSplit(n_splits=1,
                        test_size=196,
                        random_state=0)
results = cross_validate(hp_model ,
                        Auto.drop(['mpg'], axis=1),
                        Auto['mpg'],
                        cv=validation);
results['test_score']

array([23.61661707])

In [20]:
validation = ShuffleSplit(n_splits=10,
                        test_size=196,
                        random_state=0)
results = cross_validate(hp_model ,
                        Auto.drop(['mpg'], axis=1),
                        Auto['mpg'],
                        cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091831)

# 5.3.3 The Bootstrap

In [21]:
Portfolio = load_data("Portfolio")
Portfolio

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983
...,...,...
95,0.479091,1.454774
96,-0.535020,-0.399175
97,-0.773129,-0.957175
98,0.403634,1.396038


In [24]:
np.cov(Portfolio[["X", "Y"]].loc[1])

array(0.22935297)

In [25]:
# returns an estimate for alpha
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) /
            (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [26]:
# using all 100 rows
alpha_func(Portfolio , range(100))

0.57583207459283

In [27]:
# randomly selecting 100 samples
rng = np.random.default_rng(0)
alpha_func(Portfolio ,
            rng.choice(100,
                        100,
                        replace=True))

0.6074452469619004

In [28]:
# a generalized function
def boot_SE(func ,
            D,
            n=None ,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_ , second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index ,
                        n,
                        replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [29]:
# using 1000 bootstrap repetitions
alpha_SE = boot_SE(alpha_func ,
                    Portfolio ,
                    B=1000,
                    seed=0)
alpha_SE

0.09118176521277699

## Estimating the accuracy of a Linear Regression model

In [30]:
# function to bootstrap a regression model
def boot_OLS(model_matrix , response , D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [31]:
# freezing 1st 2 arguments using partial
hp_func = partial(boot_OLS , MS(['horsepower']), 'mpg')

In [32]:
# hp_func now has only 2 arguments
hp_func?

In [33]:
# creating estimates for B0 and B1
rng = np.random.default_rng(0)
np.array([hp_func(Auto ,
            rng.choice(392,
                        392,
                        replace=True)) for _ in range(10)])  # 10 bootstrap samples

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [34]:
# computing standard errors using our previously defined function
hp_se = boot_SE(hp_func ,
                Auto ,
                B=1000,
                seed=10)
hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64

In [35]:
hp_model.fit(Auto , Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

The difference in results obtained using bootstrap is due to the fact that is does not take into account any assumptions, and hence gives a more accurate estimate

In [36]:
# for a quadratic model
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS ,
                    quad_model ,
                    'mpg')
boot_SE(quad_func , Auto , B=1000)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [37]:
# comparing to OLS results
M = sm.OLS(Auto['mpg'],
           quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64